# Практическая задача по вычислительной математике
## Иванов Кирилл, 625 группа, вариант 5

Импортируем необходимые библиотеки:

In [67]:
import numpy as np 
import scipy.linalg as sla
import math
import matplotlib.pyplot as plt

%matplotlib inline

#Функция для отображения вывода результата вычислений Python в формате LaTeX
from IPython.display import display, Math, Latex
def print_math(string):
    display(Math(string))

## Формулировка задачи

$$\dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) + \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = - \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right),$$
$$ x \in (0, 1), y \in (0, 1)$$

$u(0, y) = \sqrt{\sin\left(\dfrac{\pi y}{2}\right)}$ 

$u(1, y) = \sqrt{\dfrac12 \sin\left(\dfrac{\pi y}{2}\right)}$

$u(x, 0) = 0 $ 

$u(x, 1) = \sqrt{\sin\left(\dfrac{\pi x}{3}\right)}$

### Аналитическое решение задачи

Замена $W = u^2$, при которой исходное уравнение принимает вид:

$$ \Delta W = - \dfrac{13\pi^2}{36}\cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) $$

Ищем решение в виде $W = C \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)$. Подстановка дает: 

$$ C \cdot \left(-\dfrac{\pi^2}{9}\right) \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) + C \cdot \left(-\dfrac{\pi^2}{4}\right)  \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right) = - \dfrac{13\pi^2}{36} \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right), $$

$$ C \left( \dfrac19 + \dfrac14 \right) = \dfrac{13}{36} \Rightarrow C \cdot \dfrac{13}{36} = \dfrac{13}{36} \Rightarrow C = 1 $$

Получаем $W = \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)$, откуда исходная функция

$$ u =  \sqrt{ \cos\left(\dfrac{\pi x}{3}\right) \sin\left(\dfrac{\pi y}{2}\right)}$$

Вычислим истинные значения в узлах сеточной функции.

In [296]:
true = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        true[row][col] = math.sqrt(math.cos( (math.pi * row * h) / 3) * math.sin( (math.pi * col * h) / 2))
        
for col in true:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.5498 0.7582 0.8896 0.9645 0.989 ]
[0.     0.5313 0.7328 0.8597 0.9321 0.9558]
[0.     0.5    0.6896 0.809  0.8772 0.8995]
[0.     0.4547 0.6271 0.7358 0.7977 0.818 ]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


### Разностная схема

Аппроксимируем производные по схеме "крест", сетка с одинаковым по обоим измерениям шагом $ h $, узловые точки $u_{m,n}$ ($n, m$ пробегают от 0 до $N = \dfrac{1}{h}$). 

Обозначим $ \dfrac{\partial}{\partial x}\left(u \dfrac{\partial u}{\partial x}\right) = u_x, \; \dfrac{\partial}{\partial y}\left(u \dfrac{\partial u}{\partial y}\right) = u_y $, аппроксимация дает:

$$ u_x \approx \dfrac{1}{h}\left(u_{m+0.5,n} \cdot \dfrac{u_{m+1,n} - u_{m,n}}{h} - u_{m-0.5,n} \cdot \dfrac{u_{m,n} - u_{m-1,n}}{h} \right)$$

$$u_y \approx \dfrac{1}{h}\left(u_{m,n+0.5} \cdot \dfrac{u_{m,n+1} - u_{m,n}}{h} - u_{m,n-0.5} \cdot \dfrac{u_{m,n} - u_{m,n-1}}{h} \right)$$

Точки в половинных узлах аппроксимируем средним арифметическим значений в ближайших узлах. Например, $u_{m+0.5,n} = \dfrac{u_{m,n} + u_{m+1,n}}{2}$. Получаем:

$$ u_x \approx \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right)$$

$$ u_y \approx \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right)$$

Отсюда наша разностная задача: 
    
$$ \dfrac{1}{2h^2}\left((u_{m+1,n})^2 + (u_{m-1,n})^2 - 2(u_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u_{m,n+1})^2 + (u_{m,n-1})^2 - 2(u_{m,n})^2 \right) = - \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right), $$

Где $x_m = hm$, $y_n = hn$. Граничные условия получаем в виде

$u_{m, 0} = \sqrt{\sin\left(\dfrac{\pi y_n}{2}\right)}$ 

$u_{m, N} = \sqrt{\dfrac12 \sin\left(\dfrac{\pi y_n}{2}\right)}$

$u_{0, n} = 0 $ 

$u_{N, n} = \sqrt{\sin\left(\dfrac{\pi x_m}{3}\right)}$

### Нестационарная задача

Рассмотрим нестационарную задачу: 

$$\dfrac{\partial u}{\partial t} = u_x + u_y + \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$$

Ее решение при достаточно большом $t$ сходится к решению нестационарной задачи. 

Решим нестационарную задачу $\textbf{методом простых итераций}$. Для этого аппроксимируем производную по времени: 

$\dfrac{\partial u}{\partial t} = \dfrac{u^{s+1} - u^s}{\tau}$

Здесь $s$ – узел сетки по времени и одновременно порядок итерации, $\tau$ – временной шаг.

Таким образом, нестационарная разностная задача:
    
$$\dfrac{u^{s+1}_{m,n} - u^s_{m,n}}{\tau} = \dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^s_{m,n})^2 \right) + \dfrac{13\pi^2}{32}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$$

C описанными выше граничными условиями.

Выражение для «нового» значения в узле $(m,n)$:

$u^{s+1}_{m,n} = u^s_{m,n} - \tau \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) + \dfrac{\tau}{2h^2}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

### Численное решение задачи на языке Python 3

Шаг сетки выберем равным по условию $h_x = h_y = h = 0.2$. 

In [320]:
h = 0.2

Вычислим $\lambda_{min}$ и $\lambda_{max}$ – минимальное и максимальное собственные значения оператора Лапласа. В дальнейшем они будут использованы для выбора $\tau$ и количества итераций метода. С учетом того, что длины отрезков по $ x $ и по $ y $ равны ($ l_x = l_y = 1 $), получаем:

$\lambda_{min} = \dfrac{\pi^2}{l_x} + \dfrac{\pi^2}{l_y} = 2\pi^2$

$\lambda_{max} = \dfrac{4}{h_x^2}+\dfrac{4}{h_y^2}-\dfrac{\pi^2}{l_1} - \dfrac{\pi^2}{l_2} = \dfrac{8}{h^2} - 2\pi^2$

Численно:

In [312]:
lambda_min = 2 * math.pi * math.pi
print_math(r'\lambda_{min} = %s' %lambda_min)
lambda_max = (8 / h ** 2.) - 2 * math.pi
print_math(r'\lambda_{max} = %s' %lambda_max)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [313]:
tau_max = 2 / lambda_max
tau = 2 / (lambda_min + lambda_max)
print_math(r"Метод \; простых \; итераций \; сходится \;, если \; \tau < \dfrac{2}{\lambda_{max}}< %s" %tau_max)
print_math(r"Выберем \; \tau = \dfrac{2}{\lambda_{min} + \lambda_{max}} = %s" % tau)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Количество итераций $S$, необходимых для обеспечения сходимости с заданной точностью $\varepsilon = 0.01$, равно: 

$S = \dfrac{\ln\varepsilon}{\ln\rho}$, где $\rho = \dfrac{1 - \xi}{1 + \xi}$, $\xi = \dfrac{\lambda_{min}}{\lambda_{max}}$

Вычислим число итераций:

In [314]:
epsilon = 0.01
xi = lambda_min / lambda_max
rho = (1 - xi) / (1 + xi)
S = math.log(epsilon) / math.log(rho)
print_math(r"\xi = %s \\ \rho = %s \\ S = %s" % (xi, rho, S))

<IPython.core.display.Math object>

In [98]:
total = int(S) + 1
print_math("Установим \; количество \; итераций \; \mathrm{total} = %s > S" %total)

<IPython.core.display.Math object>

Подсчитаем теперь итерации:

Функция для подсчета правой части, обозначенную для удобства $ R = -\dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) $ в узле $ (m, n) $:

In [252]:
def get_right(m, n):
    result = (- 13 * math.pi ** 2 / 72) * math.cos(math.pi * h * m / 3) * math.sin(math.pi * h * n / 2)
    return result

In [366]:
print_math(r'В \; узле \; (1, 1) \; \left.R\right|_{m, n = 1} = %s' % get_right(1, 1))

<IPython.core.display.Math object>

Теперь выполним подсчет«нового» значения в узле $ (m, n) $.

Пусть $U = \left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2 - 4(u^s_{m,n})^2 \right)$

$ \widetilde{R} = u^s_{m, n} - \tau  R + \dfrac{\tau}{2h^2} U $

Таким образом, наша функция:

In [254]:
def get_new(old, m, n):
    combination = (old[m - 1][n]) ** 2 + (old[m + 1][n]) ** 2 + (old[m][n - 1]) ** 2 + (old[m][n + 1]) ** 2
    result = old[m][n] - tau * get_right(m, n) + (tau / (2 * h ** 2)) * (combination - 4 * (old[m][n]) ** 2)
    
    return result

Теперь проведем итерации для нулевых начальных значений во всех узлах с учетом граничных условий.

In [321]:
#Создадим массивы:
size = int(1 / h + 1)
old = np.zeros((size, size))
set_values = np.zeros((size, size))

#Введем начальные значения:
for row in range(size):
    old[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
    old[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
    old[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))

Таким образом, после начальных условий получаем:

In [256]:
np.set_printoptions(precision=4)
for col in old:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.     0.     0.     0.     0.9752]
[0.     0.     0.     0.     0.     0.8995]
[0.     0.     0.     0.     0.     0.7667]
[0.     0.     0.     0.     0.     0.5559]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


Теперь проведем итерации:

In [372]:
for iteration in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            set_values[row][col] = get_new(old, row, col)
    
    old = set_values
    for row in range(size):
        old[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
        old[0][row] = math.sqrt(math.sin(math.pi * h * row / 2)); 
        old[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for col in set_values:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.2892 0.4743 0.6429 0.8234 0.9752]
[0.     0.0656 0.1588 0.3448 0.6568 0.8995]
[0.     0.0402 0.0911 0.2189 0.5308 0.7667]
[0.     0.1586 0.2741 0.3824 0.5187 0.5559]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


Сравним полученные значения, вычитая истинные значения из приближенных:

In [373]:
r_mpi = true - set_values
for col in r_mpi:
    print(col)

[0. 0. 0. 0. 0. 0.]
[0.     0.2606 0.2839 0.2466 0.1411 0.0138]
[0.     0.4657 0.574  0.5149 0.2753 0.0563]
[0.     0.4598 0.5985 0.5901 0.3464 0.1328]
[0.     0.2961 0.353  0.3534 0.279  0.2621]
[0.0000e+00 0.0000e+00 1.1102e-16 0.0000e+00 1.1102e-16 0.0000e+00]


Видно, что аппроксимация прошла успешно.

### Метод Якоби

Рассмотрим разностную задачу: 

$\dfrac{1}{2h^2}\left((u^s_{m+1,n})^2 + (u^s_{m-1,n})^2 - 2(u^{s+1}_{m,n})^2 \right) + \dfrac{1}{2h^2}\left((u^s_{m,n+1})^2 + (u^s_{m,n-1})^2 - 2(u^{s+1}_{m,n})^2 \right) = -\dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right)$

$ s $ – порядок итерации. 

Выражение для «нового» значения в узле $(m, n)$:

$(u^{s+1}_{m,n})^2 = \dfrac{h^2}{2} \dfrac{13\pi^2}{72}\cos\left(\dfrac{\pi x_m}{3}\right) \sin\left(\dfrac{\pi y_n}{2}\right) + \dfrac{1}{4}\left((u^s_{m-1,n})^2 + (u^s_{m+1,n})^2 + (u^s_{m,n-1})^2 + (u^s_{m,n+1})^2\right)$

Проведем аналогичные численные вычисления для нашей задачи. Число итераций и шаг сетки берем те же, что при МПИ.

Выполним подсчет«нового» значения в узле $ (m, n) $:

In [377]:
# подсчет нового значения в узле (m, n)

def get_Jacoby(old, m, n):
    combination = (old[m - 1][n]) ** 2 + (old[m + 1][n]) ** 2 + (old[m][n - 1]) ** 2 + (old[m][n + 1]) ** 2
    result = combination / 4 - (h ** 2 / 2) * get_right(m, n)
    
    return result

In [378]:
# реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
old_j = np.zeros((size, size))
jacoby = np.zeros((size, size))

for row in range(size):
    old_j[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
    old_j[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
    old_j[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))

for iteration in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            jacoby[row][col] = math.sqrt(get_Jacoby(old_j, row, col))
    
    old_j = jacoby
    for row in range(size):
        old_j[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
        old_j[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
        old_j[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for col in jacoby:
    print(col)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.5442 0.7492 0.8766 0.9481 0.9752]
[0.     0.5207 0.7149 0.8317 0.89   0.8995]
[0.     0.4868 0.6661 0.768  0.8033 0.7667]
[0.     0.4445 0.6076 0.6968 0.7082 0.5559]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


$\textbf{8. Сравнение численного и аналитического решений}$

Сравним полученные значения, вычитая истинные значения из приближенных:

In [379]:
rj = true - jacoby
for col in rj:
    print(col)

[0. 0. 0. 0. 0. 0.]
[0.     0.0056 0.009  0.013  0.0164 0.0138]
[0.     0.0107 0.0179 0.028  0.0421 0.0563]
[0.     0.0132 0.0235 0.041  0.0739 0.1328]
[0.     0.0103 0.0195 0.0389 0.0895 0.2621]
[0.0000e+00 0.0000e+00 1.1102e-16 0.0000e+00 1.1102e-16 0.0000e+00]


Видно, что аппроксимация прошла успешно.

$\textbf{9. Сравнение метода простой итерации и метода Якоби}$

Сравним эвклидову норму разности истинного решения и численного по методу простых итераций $msi\_norm$ и эвклидову норму разности истинного решения и численного по методу Якоби $jacoby\_norm$. 

In [386]:
msi_n = sla.norm(true - set_values)
jacoby_n = sla.norm(true - jacoby)

print_math(r"||MSI_n|| = %s" % msi_n)
print_math(r"||J_n|| = %s" % jacoby_n)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [382]:
# подсчет правой части в точках m, n

def get_right_part(m, n):
    ans =  (- 5 * math.pi ** 2 / 9) * (math.sin(math.pi * h * m / 3) + math.cos(math.pi * h * m / 3)) * math.sin(math.pi * h * n)
    return ans

In [340]:
def get_new_value(prev_values, m, n):
    combination = (prev_values[m - 1][n]) ** 2 + (prev_values[m + 1][n]) ** 2 + (prev_values[m][n - 1]) ** 2 + (prev_values[m][n + 1]) ** 2
    ans = prev_values[m][n] - tau * get_right_part(m, n) + (tau / (2 * h * h)) * (combination - 4 * (prev_values[m][n]) ** 2)
    
    return ans

In [345]:
#реализация итераций
# начальные значения во всех узлах нулевые

size = int(1 / h + 1.)
prev_values = np.zeros((size, size))
cur_values = np.zeros((size, size))

for row in range(size):
#     prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
    prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row))
    prev_values[size - 1][row] = math.sqrt((1 + math.sqrt(3)) * 0.5 * math.sin(math.pi * h * row / 2))

for it in range(total):
    for row in range(1, size - 1):
        for col in range(1, size - 1):
            cur_values[row][col] = get_new_value(prev_values, row, col)
    
    prev_values = cur_values
    for row in range(size):
        prev_values[row][size - 1] = math.sqrt(math.cos(math.pi * h * row / 2))
    
        prev_values[0][row] = math.sqrt(math.sin(math.pi * h * row / 2))
        prev_values[size - 1][row] = math.sqrt(0.5 * math.sin(math.pi * h * row / 2))
    
for x in cur_values:
    print(x)

[0.     0.5559 0.7667 0.8995 0.9752 1.    ]
[0.     0.4006 0.6091 0.737  0.8518 0.9752]
[0.     0.2803 0.4512 0.5502 0.704  0.8995]
[0.     0.2787 0.4379 0.5058 0.6075 0.7667]
[0.     0.3483 0.5226 0.5919 0.6066 0.5559]
[0.     0.3931 0.5421 0.636  0.6896 0.7071]


In [355]:
true_values = np.zeros((size, size))

for row in range(size):
    for col in range(size):
        true_values[row][col] = math.sqrt((math.sin(math.pi * h * row / 3) + math.cos(math.pi * h * row / 3)) * math.sin(math.pi * h * col))
        
for x in true_values:
    print(x)

[0.0000e+00 7.6667e-01 9.7522e-01 9.7522e-01 7.6667e-01 1.1066e-08]
[0.0000e+00 8.3495e-01 1.0621e+00 1.0621e+00 8.3495e-01 1.2052e-08]
[0.0000e+00 8.8093e-01 1.1206e+00 1.1206e+00 8.8093e-01 1.2716e-08]
[0.0000e+00 9.0610e-01 1.1526e+00 1.1526e+00 9.0610e-01 1.3079e-08]
[0.0000e+00 9.1111e-01 1.1589e+00 1.1589e+00 9.1111e-01 1.3151e-08]
[0.0000e+00 8.9606e-01 1.1398e+00 1.1398e+00 8.9606e-01 1.2934e-08]


In [354]:
cur_values - true_values

array([[ 0.    , -0.2108, -0.2085, -0.0758,  0.2085,  1.    ],
       [ 0.    , -0.4344, -0.453 , -0.325 ,  0.0169,  0.9752],
       [ 0.    , -0.6006, -0.6694, -0.5703, -0.1769,  0.8995],
       [ 0.    , -0.6274, -0.7147, -0.6468, -0.2986,  0.7667],
       [ 0.    , -0.5628, -0.6364, -0.567 , -0.3045,  0.5559],
       [ 0.    , -0.503 , -0.5977, -0.5038, -0.2065,  0.7071]])